<a href="https://colab.research.google.com/github/Mogreine29/HandOnAI_1/blob/main/7_VisionTransformeur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vision Transformer

In [1]:
!pip3 install datasets
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 8.4 MB/s 
     |████████████████████████████████| 115 kB 60.1 MB/s 
     |████████████████████████████████| 182 kB 57.5 MB/s 
     |████████████████████████████████| 212 kB 59.9 MB/s 
     |████████████████████████████████| 127 kB 60.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 27.8 MB/s 
     |████████████████████████████████| 7.6 MB 58.2 MB/s 


In [2]:
from IPython.display import Image, HTML, display
from matplotlib import pyplot as plt
import numpy as np 
import os
import cv2
import csv

import math
import argparse
import matplotlib
import imghdr
import pickle as pkl
import datetime
from cycler import cycler
from PIL import Image, ImageEnhance

from pathlib import Path
import os.path

from datasets import load_dataset
from datasets import load_metric

from transformers import TrainingArguments
from transformers import ViTFeatureExtractor
from transformers import ViTForImageClassification

import torch

from PIL import Image
import requests
import numpy as np

## Récupération des données



## Dataset 
Récupération des données sous forme de Dataset. Les données "train" sont ensuite splitté en deux afin d'avoir des données de validation. 

Concernant les données de test. On récupère le Dataset prévu à cet effet. 

In [37]:
ds = load_dataset("imagefolder", data_dir = "./gdrive/MyDrive/Challenge1/db2")
ds

Resolving data files:   0%|          | 0/3298 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset imagefolder downloaded and prepared to /root/.cache/huggingface/datasets/imagefolder/default-62bce2400c7e11c5/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 3298
    })
})

In [38]:
data = ds['train'].train_test_split(test_size = 0.1)
data

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 2968
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 330
    })
})

Utilisation de dictionnaires afin de relier les ID à l'élément de classification. 

In [39]:
labels = data['train'].features['label']
labels = data["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [40]:
metric = load_metric('accuracy')

Chargement du feature extractor. On précise le Transformer que l'on souhaite utiliser. 

In [41]:
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

loading configuration file preprocessor_config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/1ba429d32753f33a0660b80ac6f43a3c80c18938/preprocessor_config.json
Feature extractor ViTFeatureExtractor {
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ViTFeatureExtractor",
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "size": 224
}



In [42]:
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
            RandomResizedCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

val_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

In [43]:
train_ds = data['train']
val_ds = data['test']
test_ds = load_dataset("imagefolder", data_dir = "./test")

Resolving data files:   0%|          | 0/210 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
test_ds

DatasetDict({
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 210
    })
})

In [45]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)
test_ds.set_transform(preprocess_val)

In [46]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=300x168 at 0x7FDA25348D10>,
 'label': 0,
 'pixel_values': tensor([[[-0.0275, -0.0275, -0.0275,  ...,  0.8745,  0.8431,  0.8431],
          [-0.0275, -0.0275, -0.0275,  ...,  0.8745,  0.8431,  0.8431],
          [-0.0275, -0.0275, -0.0275,  ...,  0.8824,  0.8510,  0.8510],
          ...,
          [-0.6627, -0.6627, -0.6627,  ..., -0.6549, -0.6549, -0.6549],
          [-0.6627, -0.6627, -0.6627,  ..., -0.6549, -0.6549, -0.6549],
          [-0.6627, -0.6627, -0.6627,  ..., -0.6549, -0.6549, -0.6549]],
 
         [[-0.5529, -0.5529, -0.5529,  ...,  0.5451,  0.5765,  0.5765],
          [-0.5529, -0.5529, -0.5529,  ...,  0.5451,  0.5765,  0.5765],
          [-0.5529, -0.5529, -0.5529,  ...,  0.5529,  0.5922,  0.5922],
          ...,
          [-0.6627, -0.6627, -0.6627,  ..., -0.6549, -0.6549, -0.6549],
          [-0.6627, -0.6627, -0.6627,  ..., -0.6549, -0.6549, -0.6549],
          [-0.6627, -0.6627, -0.6627,  ..., -0.6549, 

In [47]:
model_name_or_path = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(
    model_name_or_path, 
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224-in21k/snapshots/1ba429d32753f33a0660b80ac6f43a3c80c18938/config.json
Model config ViTConfig {
  "_name_or_path": "google/vit-base-patch16-224-in21k",
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "fire",
    "1": "no_fire",
    "2": "start_fire"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "fire": "0",
    "no_fire": "1",
    "start_fire": "2"
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.24.0"
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--vit-base-

In [48]:
training_args = TrainingArguments(
    'finetuned-fire-detection',
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
  hub_strategy="end"
)#

PyTorch: setting up devices


In [49]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [50]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [51]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)

Using cuda_amp half precision backend


In [52]:
train_results = trainer.train()
# rest is optional but nice to have
torch.save(model.state_dict(), "model.pth")
trainer.save_model('model.h5')
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2968
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 744
  Number of trainable parameters = 85800963


Step,Training Loss,Validation Loss,Accuracy
100,0.139000,0.133984,0.966667
200,0.130600,0.096661,0.975758
300,0.067200,0.113519,0.969697
400,0.122000,0.075900,0.978788
500,0.068400,0.054464,0.981818
600,0.041300,0.040007,0.987879
700,0.035200,0.041704,0.984848


***** Running Evaluation *****
  Num examples = 330
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag)
Saving model checkpoint to finetuned-fire-detection/checkpoint-100
Configuration saved in finetuned-fire-detection/checkpoint-100/config.json
Model weights saved in finetuned-fire-detection/checkpoint-100/pytorch_model.bin
Feature extractor saved in finetuned-fire-detection/checkpoint-100/preprocessor_config.json
Deleting older checkpoint [finetuned-fire-detection/checkpoint-200] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 330
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag)
Saving model c

***** train metrics *****
  epoch                    =         4.0
  total_flos               = 856810385GF
  train_loss               =       0.115
  train_runtime            =  0:35:10.27
  train_samples_per_second =       5.626
  train_steps_per_second   =       0.353


In [53]:
metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 330
  Batch size = 8


/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 20 bytes but only got 19. Skipping tag 36867
  " Skipping tag %s" % (size, len(data), tag)


***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.9879
  eval_loss               =       0.04
  eval_runtime            = 0:00:13.77
  eval_samples_per_second =      23.96
  eval_steps_per_second   =      3.049


## Évaluation du modèle

In [54]:
outputs = trainer.predict(test_ds['test'])
print(outputs.metrics)

***** Running Prediction *****
  Num examples = 210
  Batch size = 8


{'test_loss': 0.1880544275045395, 'test_accuracy': 0.9476190476190476, 'test_runtime': 6.7192, 'test_samples_per_second': 31.254, 'test_steps_per_second': 4.018}
